<a href="https://colab.research.google.com/github/hyunjung-kim0524/DL_Study/blob/main/20240426_3_2_DataSet_%EA%B5%AC%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

pandas를 이용한 데이터 set 생성

In [4]:
import pandas as pd

file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"

heart_df = pd.read_csv(file_url)
print("Pamdas DataFrame : ", type(heart_df))
heart_df.head()

Pamdas DataFrame :  <class 'pandas.core.frame.DataFrame'>


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


tf.data를 이용한 데이터 set 구성

In [18]:
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
from tensorflow.data.experimental import make_csv_dataset

file_name = "/content/drive/MyDrive/Colab Notebooks/data/heart.csv"

heart_ds = make_csv_dataset(file_name, batch_size = 2)

print("tf.data : ", type(heart_ds))

import numpy as np
# numpy 형태로 변환하기(선택사항)
interator = heart_ds.as_numpy_iterator()

print(dict(next(interator)))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
tf.data :  <class 'tensorflow.python.data.ops.prefetch_op._PrefetchDataset'>
{'age': array([45, 59], dtype=int32), 'sex': array([1, 1], dtype=int32), 'cp': array([1, 4], dtype=int32), 'trestbps': array([110, 140], dtype=int32), 'chol': array([264, 177], dtype=int32), 'fbs': array([0, 0], dtype=int32), 'restecg': array([0, 0], dtype=int32), 'thalach': array([132, 162], dtype=int32), 'exang': array([0, 1], dtype=int32), 'oldpeak': array([1.2, 0. ], dtype=float32), 'slope': array([2, 1], dtype=int32), 'ca': array([0, 1], dtype=int32), 'thal': array([b'reversible', b'reversible'], dtype=object), 'target': array([0, 1], dtype=int32)}


pandas 데이터 프레임을 tf.data.Dataset 으로 변환하기

In [28]:
# copy()로 복사하기
df = heart_df.copy()

# pop함수로 label과 피쳐 구분하기 --> target열은 label에 저장되고 df 데이터 프레임에서 target열은 삭제됨
label = df.pop('target')

# tf.data.Dataset의 from_tensor_slices 함수를 사용해서 tf.data.Dataset 으로 데이터 프레임 변경
ds = tf.data.Dataset.from_tensor_slices((dict(df), label))
ds = ds.batch(3)  # 배치단위 설정

print("tf.data : ", type(ds))

print(list(ds.as_numpy_iterator()))


#for batch in ds :
#  print(batch)

tf.data :  <class 'tensorflow.python.data.ops.batch_op._BatchDataset'>
[({'age': array([63, 67, 67]), 'sex': array([1, 1, 1]), 'cp': array([1, 4, 4]), 'trestbps': array([145, 160, 120]), 'chol': array([233, 286, 229]), 'fbs': array([1, 0, 0]), 'restecg': array([2, 2, 2]), 'thalach': array([150, 108, 129]), 'exang': array([0, 1, 1]), 'oldpeak': array([2.3, 1.5, 2.6]), 'slope': array([3, 2, 2]), 'ca': array([0, 3, 2]), 'thal': array([b'fixed', b'normal', b'reversible'], dtype=object)}, array([0, 1, 0])), ({'age': array([37, 41, 56]), 'sex': array([1, 0, 1]), 'cp': array([3, 2, 2]), 'trestbps': array([130, 130, 120]), 'chol': array([250, 204, 236]), 'fbs': array([0, 0, 0]), 'restecg': array([0, 2, 0]), 'thalach': array([187, 172, 178]), 'exang': array([0, 0, 0]), 'oldpeak': array([3.5, 1.4, 0.8]), 'slope': array([3, 1, 1]), 'ca': array([0, 0, 0]), 'thal': array([b'normal', b'normal', b'normal'], dtype=object)}, array([0, 0, 0])), ({'age': array([62, 57, 63]), 'sex': array([0, 0, 1]), 'cp'

이미지 파일을 Dataset으로 불러오기